# VECTOR TILING FOR MAP GENERALIZATION

Generalization means creating many copies of a single ‘source of truth’ and tailoring them to the view context

Doing this on the fly is labor intensive in a static paper mapping context, especially en masse with variable map scales and contexts (urban, semi-urban, rural) 

Value in adopting dynamic mapping techniques into the existing processing workflow:

- Vector tiling: highly optimized, customizable, retains records’ attributes (versus image tiles)
- Customization, filtering, alignment between 'in-house' GRID3 data and third-party sources like OpenStreetMap
- API endpoint targets: maps can diff and grow as the sources of truth do
- TIPPECANOE: Incredible and fast processing library, somewhat unpredictable w/ emergent effects when processing geometries in a ‘recursive tiled’ manner, but benefits outweigh headaches
- Plus: Self-hosting interest (docker/containerization): anxiety after ‘open’ data portals going offline nationwide



<!-- - What works?

- What doesn't? -->
<!-- 
Note: “Emergent properties” -> usually helpful automation, as long as there are constraints, but need to watch for weirdness and misalignment with what a legend would otherwise say a layer/category should look like

(For instance, interpolation between color steps, etc) -->

===

# Processing pipeline

1. **Download** - Fetch Overture Maps and GRID3 (AGOL feature services) data for specified extent (as GeoParquet file)
2. **Convert to FlatGeobuf** - Transform GeoParquet to FlatGeobuf for compatibility + efficiency
3. **Tile** - Generate PMTiles using tippecanoe with bespoke settings per-layer
4. **View** - using maplibre open spec

## File formats
- **GeoParquet (.parquet)** - Download format (compact, fast queries via "duckquery")
- **FlatGeobuf (.fgb)** - Convert for optimal tippecanoe library support
- **GeoJSON (.geojson)** - Fallback support for small datasets
- **pmTiles** - Dynamic vector tiles, served from single static file

## CONFIG

.env -> config.py imports environment vars

In [2]:
# ============================================================
# CONFIGURATION - Run this cell first
# ============================================================
# This cell initializes all configuration and should be run 
# first. Re-run this cell to reload configuration changes.
# ============================================================

import sys
import os
from pathlib import Path
from dotenv import load_dotenv

# Setup paths
notebook_dir = Path.cwd()
processing_dir = notebook_dir.parent  # 1-processing
repo_root = processing_dir.parent     # basemap (repository root)

# Add processing directory to path
if str(processing_dir) not in sys.path:
    sys.path.insert(0, str(processing_dir))

# Load environment variables from REPOSITORY ROOT (monorepo-wide .env)
env_path = repo_root / '.env'
load_dotenv(env_path)
print(f"✓ Loaded environment from repository root: {env_path}")
print(f"  DATA_DISK = {os.environ.get('DATA_DISK', 'not set')}")

# Import configuration (will also load .env via config.py)
from config import (
    get_config,
    ensure_directories,
    print_config_summary,
    SCRIPTS_DIR,
    OUTPUT_DIR,
    OVERTURE_DATA_DIR,
    GRID3_DATA_DIR,
    SCRATCH_DIR,
)

# Import processing functions
from scripts import (
    download_overture_data,
    convert_file,
    convert_parquet_to_fgb,
    batch_convert_directory,
    process_to_tiles,
    create_tilejson,
    download_arcgis_data,
    batch_download_arcgis_layers,
)

# Additional libraries
import pandas as pd
import polars as pl
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# GLOBAL CONFIGURATION - Available in all cells below
# ============================================================
CONFIG = get_config()

# ============================================================
# EXTENT CONFIGURATION - SINGLE SOURCE OF TRUTH
# ============================================================
# Extent is now configured in .env file (repository root)
# To change the geographic area, edit .env and restart kernel
# 
# Current extent values from .env:
print(f"\n=== EXTENT FROM ENVIRONMENT (.env) ===")
print(f"  West (lon_min):  {os.environ.get('EXTENT_WEST', 'not set')}")
print(f"  South (lat_min): {os.environ.get('EXTENT_SOUTH', 'not set')}")
print(f"  East (lon_max):  {os.environ.get('EXTENT_EAST', 'not set')}")
print(f"  North (lat_max): {os.environ.get('EXTENT_NORTH', 'not set')}")
print(f"  Buffer (degrees): {os.environ.get('EXTENT_BUFFER', 'not set')}")
print(f"\n  Combined tuple: {CONFIG['extent']['coordinates']}")
print(f"  Buffer: {CONFIG['extent']['buffer_degrees']} degrees")

# DO NOT override extent here - edit .env instead!
# CONFIG["extent"]["coordinates"] is automatically loaded from .env

# ============================================================
# DISK SPACE MANAGEMENT
# ============================================================
# Automatically remove source files after successful conversion to save disk space
# FlatGeobuf (.fgb) files are kept as intermediary files for fast PMTiles regeneration
CONFIG["fgb_conversion"]["cleanup_source"] = True  # Set to False to keep source files

# Processing options (can still be customized here)
CONFIG["tiling"]["input_dirs"] = [SCRATCH_DIR]  # Read FlatGeobuf files from scratch
CONFIG["download"]["verbose"] = True
CONFIG["conversion"]["verbose"] = True
CONFIG["tiling"]["verbose"] = True
CONFIG["tiling"]["parallel"] = True

# Create directories and verify
ensure_directories()

# Verification
print("\n=== CONFIGURATION VERIFICATION ===")
print(f"Repository root:       {repo_root}")
print(f"Environment .env:      {env_path}")
print(f"Environment DATA_DISK: {os.environ.get('DATA_DISK', 'NOT SET')}")
print(f"Config uses:           {CONFIG['paths']['data_dir'].parent}")

print_config_summary(CONFIG)

# Disk management info
print("\n=== DISK SPACE MANAGEMENT ===")
print(f"  Cleanup source files: {CONFIG['fgb_conversion']['cleanup_source']}")
if CONFIG['fgb_conversion']['cleanup_source']:
    print("  → Source .parquet files will be removed after successful .fgb conversion")
    print("  → FlatGeobuf (.fgb) files retained for fast PMTiles regeneration")
else:
    print("  → Both .parquet and .fgb files will be kept")

print("\n✓ Configuration loaded - CONFIG available in all cells")
print("\n⚠️  To change extent: Edit .env file and restart kernel")
print("   All downloads, processing, and viewing will use the same extent")

✓ Loaded environment from repository root: c:\Users\mjh2241\Documents\GitHub\.env
  DATA_DISK = .

=== EXTENT FROM ENVIRONMENT (.env) ===
  West (lon_min):  12.19
  South (lat_min): -13.46
  East (lon_max):  31.31
  North (lat_max): 5.39
  Buffer (degrees): 0.5

  Combined tuple: (12.19, -13.46, 31.31, 5.39)
  Buffer: 0.5 degrees

=== CONFIGURATION VERIFICATION ===
Repository root:       c:\Users\mjh2241\Documents\GitHub
Environment .env:      c:\Users\mjh2241\Documents\GitHub\.env
Environment DATA_DISK: .
Config uses:           C:\Users\mjh2241\Documents\GitHub\mapTiles
PROJECT CONFIGURATION
Project root:        C:\Users\mjh2241\Documents\GitHub\mapTiles\1-processing
Scripts directory:   C:\Users\mjh2241\Documents\GitHub\mapTiles\1-processing\scripts
Notebooks directory: C:\Users\mjh2241\Documents\GitHub\mapTiles\1-processing\notebooks
Data directory:      C:\Users\mjh2241\Documents\GitHub\mapTiles\data
Scratch directory:   C:\Users\mjh2241\Documents\GitHub\mapTiles\data\2-scratch
Out

c:\Users\mjh2241\Documents\GitHub\mapTiles\1-processing\scripts\generateLabels.py:37: UserWarning: centerline library not available. Install with: pip install centerline
  warnings.warn("centerline library not available. Install with: pip install centerline")
c:\Users\mjh2241\Documents\GitHub\mapTiles\1-processing\scripts\xgboost_optimizer.py:31: UserWarning: xgboost not available. Install with: pip install xgboost
  warnings.warn("xgboost not available. Install with: pip install xgboost")


## 2. Download Overture Data with DuckDB 

Use the `downloadOverture.py` module to fetch geospatial data from Overture Maps

e.g., replace periodic geofabrik OSM ~shapefile~ fetches

In [ ]:
# Download Overture Maps data
print("=== STEP 1: DOWNLOADING OVERTURE DATA ===")
download_results = download_overture_data(
    extent=CONFIG["extent"]["coordinates"],
    buffer_degrees=CONFIG["extent"]["buffer_degrees"],
    template_path=str(CONFIG["paths"]["template_path"]),
    verbose=CONFIG["download"]["verbose"],
    project_root=str(CONFIG["paths"]["project_root"]),
    overture_data_dir=str(CONFIG["paths"]["overture_data_dir"])
)

print(f"Download completed: {download_results['success']}")
print(f"Sections processed: {download_results['processed_sections']}")
if download_results["errors"]:
    print(f"Errors encountered: {len(download_results['errors'])}")
    for error in download_results["errors"]:
        print(f"  - {error}")
print()

## Overture Maps Cartography Properties

The updated DuckDB queries now extract **cartography** and **level** properties from Overture Maps data (2025-11-19.0 release):

### Cartography Container (base theme layers)
Available in: `land_cover`, `land_use`, `water`, `land`, `infrastructure`

- **`min_zoom`** - Recommended minimum zoom level for displaying this feature
- **`max_zoom`** - Recommended maximum zoom level for displaying this feature  
- **`sort_key`** - Z-ordering priority for rendering (higher values draw on top)

### Level Property
Available in: Most base theme layers, buildings, infrastructure

- **`level`** - Vertical level/floor number for multi-level features
- Used for proper z-ordering in buildings and infrastructure
- Enables floor-aware filtering and visualization

### How These Properties Are Used

**1. During Tiling (`tippecanoe.py`):**
- `extract_cartography_zoom_range()` reads cartography properties from input files
- Automatically applies optimal zoom ranges to each layer
- `sort_key` and `level` are preserved in PMTiles for renderer use

**2. In the Map Viewer (`basemap.js`):**
- Properties are available as feature attributes in PMTiles
- Can be used in MapLibre style expressions for:
  - Zoom-dependent filtering: `[">=", ["zoom"], ["get", "min_zoom"]]`
  - Z-ordering: `["get", "sort_key"]` in `symbol-sort-key`
  - Floor filtering: `["==", ["get", "level"], 0]`
  - Dynamic styling based on feature importance

**3. Benefits:**
- More efficient tiles (features only at appropriate zoom levels)
- Better rendering performance (proper z-ordering reduces overdraw)
- Richer data model (multi-level buildings, elevation-aware features)
- Aligned with Overture's cartographic recommendations

### What Changed in the Pipeline

**✅ Updated: `tileQueries.sql`**
- Added `level` property to `land_cover` query (was missing)
- Added cartography properties (`min_zoom`, `max_zoom`, `sort_key`) to:
  - `land_use` (both residential and non-residential)
  - `water`
  - `land`
  - `infrastructure`

**✅ Updated: `tippecanoe.py`**
- Enhanced documentation for `extract_cartography_zoom_range()` to mention level and sort_key
- Updated `build_tippecanoe_command()` docstring to explain how properties are preserved

**✅ No changes needed:**
- `convertToFlatGeobuf.py` - Already preserves all attributes automatically
- `downloadOverture.py` - No changes needed (uses SQL template)
- Conversion pipeline - Properties flow through automatically

**Result:** All Overture Maps cartography properties are now extracted, preserved, and available in your PMTiles for use in MapLibre styles!

## 2a. Download ArcGIS Feature Server Data

Download geospatial data from hosted ArcGIS Feature Server REST API endpoints - can include any esri-hosted data as endpoint

- **Automatic pagination** - Handles ArcGIS's 1000-2000 feature limit per request
- **Spatial filtering** - Apply bounding box filter to download only features in aoi
- **formats** - Download as GeoJSON or directly convert to FlatGeobuf
- **Batch processing** - Download multiple layers with one function call

### GRID3 DRC Layers
- https://services3.arcgis.com/BU6Aadhn6tbBEdyk/arcgis/rest/services

In [2]:
# Download ArcGIS Feature Server data (optional - skip if not needed)
print("=== STEP 2a: DOWNLOADING ARCGIS DATA (OPTIONAL) ===")
print(f"Using extent from CONFIG: {CONFIG['extent']['coordinates']}")
print(f"  Longitude: {CONFIG['extent']['coordinates'][0]} to {CONFIG['extent']['coordinates'][2]}")
print(f"  Latitude: {CONFIG['extent']['coordinates'][1]} to {CONFIG['extent']['coordinates'][3]}")

# Define ArcGIS layers to download
# Uncomment and customize the layers you need
arcgis_layers = [
    {
        'url': 'https://services3.arcgis.com/BU6Aadhn6tbBEdyk/arcgis/rest/services/GRID3_COD_Settlement_Extents_v3_1/FeatureServer/0',
        'name': 'settlement_extents',
        'where': '1=1'
    },
    {
        'url': 'https://services3.arcgis.com/BU6Aadhn6tbBEdyk/arcgis/rest/services/GRID3_COD_health_zones_v7_0/FeatureServer/0',
        'name': 'health_zones',
        'where': '1=1'  # Download all features (can add SQL filter here)
    },
    {
        'url': 'https://services3.arcgis.com/BU6Aadhn6tbBEdyk/ArcGIS/rest/services/GRID3_COD_health_areas_v7_0/FeatureServer/0',
        'name': 'health_areas',
        'where': '1=1'
    },
        {
        'url': 'https://services3.arcgis.com/BU6Aadhn6tbBEdyk/ArcGIS/rest/services/GRID3_COD_settlement_names_v7_0/FeatureServer/0',
        'name': 'settlement_names',
        'where': '1=1'  
    },
    {
        'url': 'https://services3.arcgis.com/BU6Aadhn6tbBEdyk/ArcGIS/rest/services/COD_GRID3_health_facilities_v7_0/FeatureServer/0',
        'name': 'health_facilities',
        'where': '1=1'
    },
    {
        'url': 'https://services3.arcgis.com/BU6Aadhn6tbBEdyk/ArcGIS/rest/services/GRID3_COD_religious_centers_v1_0/FeatureServer/0',
        'name': 'religious_centers',
        'where': '1=1'
    },
]

# Download layers using the SAME EXTENT as Overture data
# This ensures all data layers align spatially
if arcgis_layers:
    arcgis_results = batch_download_arcgis_layers(
        layer_configs=arcgis_layers,
        output_dir=str(CONFIG["paths"]["scratch_dir"]),  # Save directly to scratch for tiling
        extent=CONFIG["extent"]["coordinates"],  # ← SAME EXTENT as Overture downloads
        output_format="fgb",  # Use FlatGeobuf for optimal tiling performance
        verbose=CONFIG["download"]["verbose"]
    )
    
    print(f"\nArcGIS Download Summary:")
    print(f"  Total layers: {arcgis_results['total_layers']}")
    print(f"  Successful: {arcgis_results['successful']}")
    print(f"  Failed: {arcgis_results['failed']}")
    
    for layer in arcgis_results['layers']:
        if layer['success']:
            print(f"  ✓ {layer['name']}: {layer['feature_count']:,} features")
        else:
            print(f"  ✗ {layer['name']}: {layer.get('error', 'Unknown error')}")
else:
    print("No ArcGIS layers configured. Edit arcgis_layers list above to download data.")
    print(f"\nNote: When configured, downloads will use extent: {CONFIG['extent']['coordinates']}")

# Option 2: Download a single layer (alternative approach)
# Also uses the same extent from CONFIG
# Uncomment and customize as needed:
# single_layer_result = download_arcgis_data(
#     service_url='https://services3.arcgis.com/BU6Aadhn6tbBEdyk/arcgis/rest/services/GRID3_COD_health_zones_v7_0/FeatureServer/0',
#     output_path=str(CONFIG["paths"]["scratch_dir"] / "health_zones.fgb"),
#     extent=CONFIG["extent"]["coordinates"],  # ← Uses CONFIG extent
#     output_format="fgb",
#     verbose=True
# )

=== STEP 2a: DOWNLOADING ARCGIS DATA (OPTIONAL) ===
Using extent from CONFIG: (12.19, -13.46, 31.31, 5.39)
  Longitude: 12.19 to 31.31
  Latitude: -13.46 to 5.39

Processing: settlement_extents
  URL: https://services3.arcgis.com/BU6Aadhn6tbBEdyk/arcgis/rest/services/GRID3_COD_Settlement_Extents_v3_1/FeatureServer/0
  Extent: (12.19, -13.46, 31.31, 5.39)
  Output: C:\Users\mjh2241\Documents\GitHub\mapTiles\data\2-scratch\settlement_extents.fgb
  Format: fgb
Total features to download: 0


Downloaded 0 features

Processing: health_zones
  URL: https://services3.arcgis.com/BU6Aadhn6tbBEdyk/arcgis/rest/services/GRID3_COD_health_zones_v7_0/FeatureServer/0
  Extent: (12.19, -13.46, 31.31, 5.39)
  Output: C:\Users\mjh2241\Documents\GitHub\mapTiles\data\2-scratch\health_zones.fgb
  Format: fgb


Total features to download: 0


Downloaded 0 features

Processing: health_areas
  URL: https://services3.arcgis.com/BU6Aadhn6tbBEdyk/ArcGIS/rest/services/GRID3_COD_health_areas_v7_0/FeatureServer/0
  Extent: (12.19, -13.46, 31.31, 5.39)
  Output: C:\Users\mjh2241\Documents\GitHub\mapTiles\data\2-scratch\health_areas.fgb
  Format: fgb
Total features to download: 0


Downloaded 0 features

Processing: settlement_names
  URL: https://services3.arcgis.com/BU6Aadhn6tbBEdyk/ArcGIS/rest/services/GRID3_COD_settlement_names_v7_0/FeatureServer/0
  Extent: (12.19, -13.46, 31.31, 5.39)
  Output: C:\Users\mjh2241\Documents\GitHub\mapTiles\data\2-scratch\settlement_names.fgb
  Format: fgb


Total features to download: 0


Downloaded 0 features

Processing: health_facilities
  URL: https://services3.arcgis.com/BU6Aadhn6tbBEdyk/ArcGIS/rest/services/COD_GRID3_health_facilities_v7_0/FeatureServer/0
  Extent: (12.19, -13.46, 31.31, 5.39)
  Output: C:\Users\mjh2241\Documents\GitHub\mapTiles\data\2-scratch\health_facilities.fgb
  Format: fgb
Total features to download: 0


Downloaded 0 features

Processing: religious_centers
  URL: https://services3.arcgis.com/BU6Aadhn6tbBEdyk/ArcGIS/rest/services/GRID3_COD_religious_centers_v1_0/FeatureServer/0
  Extent: (12.19, -13.46, 31.31, 5.39)
  Output: C:\Users\mjh2241\Documents\GitHub\mapTiles\data\2-scratch\religious_centers.fgb
  Format: fgb
Total features to download: 22,627


Downloaded 22,627 features
Converting to FlatGeobuf...
✓ Saved 22,627 features to C:\Users\mjh2241\Documents\GitHub\mapTiles\data\2-scratch\religious_centers.fgb

BATCH DOWNLOAD SUMMARY
Total layers: 6
Successful: 1
Failed: 5

ArcGIS Download Summary:
  Total layers: 6
  Successful: 1
  Failed: 5
  ✗ settlement_extents: No features downloaded
  ✗ health_zones: No features downloaded
  ✗ health_areas: No features downloaded
  ✗ settlement_names: No features downloaded
  ✗ health_facilities: No features downloaded
  ✓ religious_centers: 22,627 features


In [ ]:
# Generate centroids for administrative boundary labels
print("=== STEP 2b: GENERATING CENTROIDS FOR ADMINISTRATIVE LABELS ===")

# Import the centroid generation function
from scripts import batch_generate_centroids

# Define which layers need centroids for label positioning
# These correspond to the administrative boundary layers
layers_for_centroids = [
    'health_zones',   # Health zone polygons -> health_zones_centroids
    'health_areas',   # Health area polygons -> health_areas_centroids
]

# Generate centroids for specified layers
# NOTE: Centroids now include 'label_rotation' attribute for best-fit text orientation
# The rotation angle is calculated from the minimum rotated rectangle of each polygon
centroid_results = batch_generate_centroids(
    input_dir=str(CONFIG["paths"]["scratch_dir"]),  # Where polygon FGB files are
    output_dir=str(CONFIG["paths"]["scratch_dir"]),  # Save centroids alongside polygons
    layers=layers_for_centroids,                     # Only process these layers
    suffix='_centroids',                             # Output: layer_name_centroids.fgb
    verbose=CONFIG["download"]["verbose"]
)

print(f"\nCentroid Generation Summary:")
print(f"  Total layers: {centroid_results['total_layers']}")
print(f"  Successful: {centroid_results['successful']}")
print(f"  Failed: {centroid_results['failed']}")
print(f"  Note: Each centroid includes 'label_rotation' for best-fit orientation")

for layer in centroid_results['layers']:
    if layer['success']:
        output_name = Path(layer['output_file']).name
        print(f"  ✓ {output_name}: {layer['feature_count']:,} centroids")
    else:
        print(f"  ✗ {Path(layer['input_file']).name}: {layer.get('error', 'Unknown error')}")

# List all files ready for tiling (including new centroids)
if CONFIG["paths"]["scratch_dir"].exists():
    all_fgb_files = sorted(CONFIG["paths"]["scratch_dir"].glob("*.fgb"))
    centroid_files = [f for f in all_fgb_files if '_centroids' in f.name]
    
    print(f"\n{'='*50}")
    print(f"Files ready for tiling:")
    print(f"  Total FlatGeobuf files: {len(all_fgb_files)}")
    print(f"  Centroid files: {len(centroid_files)}")
    if centroid_files:
        for f in centroid_files:
            print(f"    - {f.name}")
    print(f"  Location: {CONFIG['paths']['scratch_dir']}")


## 2b. Generate Centroids for Administrative Polygons

Generate interior centroid points for health zones and health areas. These will be used for single-label-per-polygon rendering in the map viewer (interior labels at lower zoom levels).

**Why centroids?**
- Guarantees one label per polygon (no duplicates across tile boundaries)
- `representative_point()` ensures label is always inside the polygon
- Preserves all attributes for label content
- Separate point layer is more efficient than point-based symbol placement on polygons

**NEW: Label Rotation for Best-Fit**
- Each centroid now includes a `label_rotation` attribute (in degrees)
- Calculated from the minimum rotated rectangle (oriented bounding box) of each polygon
- Labels rotate to align with the polygon's longest axis
- Ensures labels fit naturally within elongated or diagonal polygons
- Rotation range: -90° to +90° (text never appears upside down)

**Map Style Integration:**
The viewer style uses:
- `"text-rotation-alignment": "map"` - Rotates with map, not viewport
- `"text-rotate": ["get", "label_rotation"]` - Uses the calculated rotation angle

## 2c. Generate Centerlines for Water Polygons

Generate centerline features for polygonal water bodies (lakes, reservoirs, etc.). These will be used for placing labels along the natural axis of elongated water features.

**Why centerlines?**
- Better label placement for elongated water features (lakes, reservoirs)
- Creates linear features along the medial axis of polygons
- Labels follow the natural orientation of the water body
- Uses Voronoi-based skeleton algorithm for accurate centerline extraction
- Preserves all attributes for label content

# temp: markdown start here
# Generate centerlines for water feature labels
print("=== STEP 2c: GENERATING CENTERLINES FOR WATER LABELS ===")

# Import the centerline generation function
from scripts import batch_generate_centerlines

# Define which layers need centerlines for label positioning
# Typically used for elongated water bodies like lakes and reservoirs
layers_for_centerlines = [
    'water',  # Water polygons -> water_centerlines
]

# Generate centerlines for specified layers
centerline_results = batch_generate_centerlines(
    input_dir=str(CONFIG["paths"]["scratch_dir"]),  # Where polygon FGB files are
    output_dir=str(CONFIG["paths"]["scratch_dir"]),  # Save centerlines alongside polygons
    layers=layers_for_centerlines,                   # Only process these layers
    suffix='_centerlines',                           # Output: layer_name_centerlines.fgb
    simplify_tolerance=5.0,                          # N meters simplification
    border_density=20,                            # Increase this for winding rivers (default is 100)
    verbose=CONFIG["download"]["verbose"]
)

print(f"  Total layers: {centerline_results['total_layers']}")
print(f"  Successful: {centerline_results['successful']}")
print(f"  Failed: {centerline_results['failed']}")

for layer in centerline_results['layers']:
    if layer['success']:
        output_name = Path(layer['output_file']).name
        print(f"  ✓ {output_name}: {layer['feature_count']:,} centerlines")
    else:
        print(f"  ✗ {Path(layer['input_file']).name}: {layer.get('error', 'Unknown error')}")

# List all processed files ready for tiling
if CONFIG["paths"]["scratch_dir"].exists():
    all_fgb_files = sorted(CONFIG["paths"]["scratch_dir"].glob("*.fgb"))
    centroid_files = [f for f in all_fgb_files if '_centroids' in f.name]
    centerline_files = [f for f in all_fgb_files if '_centerlines' in f.name]
    
    print(f"\n{'='*50}")
    print(f"Processed geometry files ready for tiling:")
    print(f"  Total FlatGeobuf files: {len(all_fgb_files)}")
    print(f"  Centroid files: {len(centroid_files)}")
    if centroid_files:
        for f in centroid_files:
            print(f"    - {f.name}")
    print(f"  Centerline files: {len(centerline_files)}")
    if centerline_files:
        for f in centerline_files:
            print(f"    - {f.name}")
    print(f"  Location: {CONFIG['paths']['scratch_dir']}")


### ArcGIS Feature Server Downloads

**Finding Feature Server URLs:**
1. Browse your organization's ArcGIS REST Services Directory
2. Navigate to a specific layer (e.g., FeatureServer/0, FeatureServer/1)
3. Copy the full URL up to and including the layer number
4. The script will automatically append `/query` and handle parameters

**Spatial Filtering:**
- The `extent` parameter filters features to your bounding box (saves bandwidth & time)
- For global layers, omit `extent=None` to download all features
- Extent uses WGS84 coordinates: `(lon_min, lat_min, lon_max, lat_max)`

**Attribute Filtering:**
- Use `where` clause for SQL-based filtering: `'population > 10000'`
- Default `'1=1'` downloads all features

**Output Formats:**
- `"fgb"` (FlatGeobuf) - Recommended for direct tiling (streaming, indexed)
- `"geojson"` - more flexible, less optimal


- Large datasets (>100k features) automatically use pagination
- Downloads directly to scratch directory

In [ ]:
# Check what files were created during download
print("=== CHECKING DOWNLOADED FILES ===")

# Check Overture Maps downloads (GeoParquet)
overture_files = []
search_dirs = [CONFIG["paths"]["data_dir"], CONFIG["paths"]["overture_data_dir"]]

for data_dir in search_dirs:
    if data_dir.exists():
        for pattern in CONFIG["download"]["output_formats"]:
            files = list(data_dir.glob(pattern))
            overture_files.extend(files)

print(f"\nOverture Maps (GeoParquet): {len(overture_files)} files")
overture_size_mb = 0
for file in sorted(overture_files):
    file_size = file.stat().st_size / 1024 / 1024  # Size in MB
    overture_size_mb += file_size
    print(f"  {file.name} ({file_size:.1f} MB)")

# Check FlatGeobuf files in scratch
fgb_files = []
if CONFIG["paths"]["scratch_dir"].exists():
    fgb_files = list(CONFIG["paths"]["scratch_dir"].glob("*.fgb"))

print(f"\nFlatGeobuf (Scratch): {len(fgb_files)} files")
fgb_size_mb = 0
for file in sorted(fgb_files):
    file_size = file.stat().st_size / 1024 / 1024  # Size in MB
    fgb_size_mb += file_size
    print(f"  {file.name} ({file_size:.1f} MB)")

# Check PMTiles
pmtiles_files = []
if CONFIG["paths"]["tile_dir"].exists():
    pmtiles_files = list(CONFIG["paths"]["tile_dir"].glob("*.pmtiles"))

print(f"\nPMTiles (Output): {len(pmtiles_files)} files")
pmtiles_size_mb = 0
for file in sorted(pmtiles_files):
    file_size = file.stat().st_size / 1024 / 1024  # Size in MB
    pmtiles_size_mb += file_size
    print(f"  {file.name} ({file_size:.1f} MB)")

# Display overall statistics
print(f"\n{'='*50}")
print(f"DISK USAGE SUMMARY")
print(f"{'='*50}")
print(f"  Overture GeoParquet: {overture_size_mb:.1f} MB ({len(overture_files)} files)")
print(f"  FlatGeobuf:          {fgb_size_mb:.1f} MB ({len(fgb_files)} files)")
print(f"  PMTiles:             {pmtiles_size_mb:.1f} MB ({len(pmtiles_files)} files)")
print(f"  Total:               {overture_size_mb + fgb_size_mb + pmtiles_size_mb:.1f} MB")

if CONFIG["fgb_conversion"]["cleanup_source"]:
    print(f"\n  Cleanup enabled: Source .parquet files will be removed after conversion")
    if overture_files:
        print(f"  Potential savings: ~{overture_size_mb:.1f} MB")
else:
    print(f"\n  Cleanup disabled: All intermediate files retained")

## 3. Convert GeoParquet to FlatGeobuf

Convert downloaded Overture GeoParquet files to FlatGeobuf format for tippecanoe compatibility

**Note**: ArcGIS data was already downloaded as FlatGeobuf in Step 2a, so this step only processes Overture Maps data. Both sources will coexist in the scratch directory.

### Disk Space Management

**Automatic cleanup enabled**: Source `.parquet` files are automatically removed after successful conversion to save disk space.

**Why this works:**
- **FlatGeobuf files (.fgb)** are retained as intermediary files
- PMTiles can be quickly regenerated from .fgb files anytime
- .fgb files are ~30-50% smaller than .parquet
- Conversion from .fgb → .pmtiles is faster than .parquet → .pmtiles

**Workflow:**
```
Download (.parquet)  →  Convert (.fgb)  →  Delete (.parquet)  →  Tile (.pmtiles)
                              ↓                                         ↑
                         [KEPT FOR]  ←──────────────────────────────────┘
                        Re-tiling anytime
```

**To disable cleanup:** Set `cleanup_source=False` in the conversion step below.

In [ ]:
# Verify that downloaded data extent matches configured extent
print("=== EXTENT VERIFICATION ===")

# Get configured extent from CONFIG (loaded from .env)
config_extent = CONFIG["extent"]["coordinates"]
config_west, config_south, config_east, config_north = config_extent

print(f"\n1. Configured Extent (from .env):")
print(f"   West:  {config_west:>10.6f}")
print(f"   South: {config_south:>10.6f}")
print(f"   East:  {config_east:>10.6f}")
print(f"   North: {config_north:>10.6f}")

# Check actual extent of downloaded files
import subprocess

def get_fgb_extent(file_path):
    """Get extent from FlatGeobuf file using ogrinfo"""
    try:
        result = subprocess.run(
            ['ogrinfo', '-al', '-so', str(file_path)],
            capture_output=True, text=True, timeout=10
        )
        for line in result.stdout.split('\n'):
            if 'Extent:' in line:
                # Parse: Extent: (20.294878, -7.704176) - (23.705435, -3.795773)
                parts = line.split(':')[1].strip()
                parts = parts.replace('(', '').replace(')', '').replace(' - ', ',')
                coords = [float(x.strip()) for x in parts.split(',')]
                return tuple(coords)  # (west, south, east, north)
    except Exception as e:
        return None
    return None

# Check key files
check_files = ['buildings.fgb', 'roads.fgb', 'water.fgb']
mismatches = []

print(f"\n2. Downloaded Data Extents:")
for filename in check_files:
    fgb_path = CONFIG["paths"]["scratch_dir"] / filename
    if fgb_path.exists():
        extent = get_fgb_extent(fgb_path)
        if extent:
            west, south, east, north = extent
            print(f"\n   {filename}:")
            print(f"     West:  {west:>10.6f}")
            print(f"     South: {south:>10.6f}")
            print(f"     East:  {east:>10.6f}")
            print(f"     North: {north:>10.6f}")
            
            # Check if extents match (within 0.5 degree tolerance for tile snapping)
            tolerance = 0.5
            west_match = abs(west - config_west) < tolerance
            south_match = abs(south - config_south) < tolerance
            east_match = abs(east - config_east) < tolerance
            north_match = abs(north - config_north) < tolerance
            
            if not (west_match and south_match and east_match and north_match):
                mismatches.append({
                    'file': filename,
                    'data_extent': extent,
                    'config_extent': config_extent
                })
    else:
        print(f"\n   {filename}: Not found")

# Report results
print(f"\n{'='*60}")
if mismatches:
    print("EXTENT MISMATCH DETECTED!")
    print(f"\n   {len(mismatches)} file(s) have different extents than configured.")
    print(f"\n   SOLUTION:")
    print(f"   1. If you want the data in the downloaded files:")
    print(f"      - Update EXTENT_* values in .env to match data extent")
    print(f"      - Restart kernel and re-run cells")
    print(f"\n   2. If you want the extent currently in .env:")
    print(f"      - Delete files: {CONFIG['paths']['scratch_dir']}")
    print(f"      - Delete files: {CONFIG['paths']['overture_data_dir']}")
    print(f"      - Re-run download cells (Step 1 and 2a)")
    print(f"\n   DO NOT proceed to tiling until extents match!")
else:
    print("EXTENT VERIFICATION PASSED")
    print(f"\n   All downloaded data matches the configured extent.")
    print(f"   Safe to proceed with tile generation.")
print(f"{'='*60}\n")


## 2.5b. Verify Extent

In [ ]:
# Convert GeoParquet files to FlatGeobuf for optimal tiling performance
print("\n" + "="*70)
print("🔄 STEP 3: CONVERTING OVERTURE GEOPARQUET TO FLATGEOBUF")
print("="*70)
print("\nℹ️  Note: ArcGIS data already in FlatGeobuf format (from Step 2a)")
print(f"🧹 Cleanup: {'ENABLED' if CONFIG['fgb_conversion']['cleanup_source'] else 'DISABLED'}")

# Get list of existing ArcGIS FlatGeobuf files to avoid overwriting
existing_fgb_files = set()
if CONFIG["paths"]["scratch_dir"].exists():
    existing_fgb_files = {f.stem for f in CONFIG["paths"]["scratch_dir"].glob("*.fgb")}
    if existing_fgb_files:
        print(f"\n✓ Preserving {len(existing_fgb_files)} existing ArcGIS FlatGeobuf files:")
        for name in sorted(list(existing_fgb_files)[:5]):  # Show first 5
            print(f"  • {name}.fgb")
        if len(existing_fgb_files) > 5:
            print(f"  ... and {len(existing_fgb_files) - 5} more")

# Convert Overture GeoParquet files to FlatGeobuf
# STREAMING ENABLED: Automatically uses Polars streaming for large files (>500MB)
# CLEANUP: Source .parquet files removed after successful conversion (saves disk space)
# The .fgb intermediary files are kept for efficient PMTiles regeneration
print("\n" + "="*70)

fgb_results = batch_convert_directory(
    input_dir=str(CONFIG["paths"]["overture_data_dir"]),
    output_dir=str(CONFIG["paths"]["scratch_dir"]),
    pattern=CONFIG["fgb_conversion"]["input_pattern"],
    overwrite=CONFIG["fgb_conversion"]["overwrite"],
    verbose=CONFIG["fgb_conversion"]["verbose"],
    cleanup_source=CONFIG["fgb_conversion"]["cleanup_source"]
)

# The module now prints a detailed summary automatically when verbose=True
# No need for duplicate summary here

# Count total FlatGeobuf files ready for tiling
if CONFIG["paths"]["scratch_dir"].exists():
    all_fgb_files = list(CONFIG["paths"]["scratch_dir"].glob("*.fgb"))
    overture_fgb_count = len([f for f in all_fgb_files if f.stem not in existing_fgb_files])
    arcgis_fgb_count = len(existing_fgb_files)
    
    print("="*70)
    print("ALL FLATGEOBUF FILES READY FOR TILING")
    print("="*70)
    print(f"  Location: {CONFIG['paths']['scratch_dir']}")
    print(f"  Total files: {len(all_fgb_files)}")
    print(f"     • Overture (converted): {overture_fgb_count}")
    print(f"     • ArcGIS (direct): {arcgis_fgb_count}")
    
    if CONFIG["fgb_conversion"]["cleanup_source"]:
        print(f"\n  ℹ  Source .parquet files removed to save disk space")
        print(f"     FlatGeobuf files retained for efficient PMTiles regeneration")
    
    # Show file listing
    print(f"\n  Files:")
    for f in sorted(all_fgb_files)[:10]:  # Show first 10
        size_mb = f.stat().st_size / 1024 / 1024
        print(f"     • {f.name} ({size_mb:.1f} MB)")
    if len(all_fgb_files) > 10:
        print(f"     ... and {len(all_fgb_files) - 10} more")
    
    print("="*70 + "\n")
else:
    print("\n No FlatGeobuf files found in scratch directory")
    if fgb_results.get('skipped', 0) > 0:
        print(f"All {fgb_results['skipped']} Overture files already converted.")

## 4. Process FlatGeobuf to PMTiles

Use the `runCreateTiles.py` module to convert FlatGeobuf files to PMTiles using custom tippecanoe queries from tippecanoe.py

In [ ]:
# Step 4: Process all geospatial files to PMTiles
print("=== STEP 4: PROCESSING TO PMTILES ===")

# Process all downloaded and converted files to PMTiles using CONFIG settings
# Now supports: GeoJSON, GeoJSONSeq, and GeoParquet formats
tiling_results = process_to_tiles(
    extent=CONFIG["extent"]["coordinates"],
    input_dirs=[str(d) for d in CONFIG["tiling"]["input_dirs"]],  # Convert Path objects to strings
    filter_pattern=CONFIG["tiling"]["filter_pattern"],  # Pass filter pattern from CONFIG
    output_dir=str(CONFIG["tiling"]["output_dir"]),  # Use explicit output directory from CONFIG
    parallel=CONFIG["tiling"]["parallel"],
    verbose=CONFIG["tiling"]["verbose"]
)

# print(f"Tiling completed: {tiling_results['success']}")
# print(f"Files processed: {len(tiling_results['processed_files'])}/{tiling_results['total_files']}")

if tiling_results["errors"]:
    print(f"Errors encountered: {len(tiling_results['errors'])}")
    for error in tiling_results["errors"]:
        print(f"  - {error}")

# Display generated PMTiles files
if tiling_results["processed_files"]:
    print(f"\n✓ Successfully generated {len(tiling_results['processed_files'])} PMTiles:")
    
    pmtiles_files = list(CONFIG["paths"]["tile_dir"].glob("*.pmtiles"))
    
    total_size_mb = 0
    for pmtile in sorted(pmtiles_files):
        size_mb = pmtile.stat().st_size / 1024 / 1024
        total_size_mb += size_mb
        print(f"  {pmtile.name} ({size_mb:.1f} MB)")
    
    print(f"\nTotal PMTiles size: {total_size_mb:.1f} MB")
    print(f"Files location: {CONFIG['paths']['tile_dir']}")
    
else:
    print("\nNo PMTiles files were generated. Check the errors above.")
    print(f"Make sure you have geospatial files (GeoJSON/GeoJSONSeq/GeoParquet) in: {[str(d) for d in CONFIG['tiling']['input_dirs']]}")

## 5. Create TileJSON Metadata for map viewer

- **Set bounds and zoom levels**
- **PMTiles URL references**

In [3]:
# Step 5: Create TileJSON metadata for MapLibre integration
print("=== STEP 5: CREATING TILEJSON METADATA ===")

# Check if PMTiles files exist in the configured tile directory
pmtiles_files = list(CONFIG["paths"]["tile_dir"].glob("*.pmtiles"))

if pmtiles_files:
    print(f"Found {len(pmtiles_files)} PMTiles files, creating TileJSON...")
    
    try:
        tilejson = create_tilejson(
            tile_dir=str(CONFIG["paths"]["tile_dir"]),  # Explicitly pass tile directory
            extent=CONFIG["extent"]["coordinates"],  # Pass extent from CONFIG
            output_file=str(CONFIG["paths"]["tile_dir"] / "tilejson.json")  # Explicitly pass output file path
        )
        
        print("✓ TileJSON created successfully")
        print(f"  Bounds: {tilejson['bounds']}")
        print(f"  Zoom range: {tilejson['minzoom']} - {tilejson['maxzoom']}")
        print(f"  Vector layers: {len(tilejson['vector_layers'])}")
        print(f"  Output file: {CONFIG['paths']['tile_dir'] / 'tilejson.json'}")
        
        # Show a summary of all output files
        print(f"\nComplete output summary:")
        total_size_mb = 0
        for pmtile in sorted(pmtiles_files):
            size_mb = pmtile.stat().st_size / 1024 / 1024
            total_size_mb += size_mb
            print(f"  {pmtile.name} ({size_mb:.1f} MB)")
        
        print(f"  tilejson.json")
        print(f"\nTotal PMTiles size: {total_size_mb:.1f} MB")
        print(f"All files location: {CONFIG['paths']['tile_dir']}")
        
    except Exception as e:
        print(f"✗ TileJSON creation failed: {e}")
        
else:
    print("No PMTiles files found in output directory.")
    print(f"Expected location: {CONFIG['paths']['tile_dir']}")
    print("Run Step 4 first to generate PMTiles files.")

=== STEP 5: CREATING TILEJSON METADATA ===
Found 6 PMTiles files, creating TileJSON...
TileJSON created: C:\Users\mjh2241\Documents\GitHub\mapTiles\data\3-pmtiles\tilejson.json
Found 6 PMTiles files
✓ TileJSON created successfully
  Bounds: [12.19, -13.46, 31.31, 5.39]
  Zoom range: 7 - 16
  Vector layers: 6
  Output file: C:\Users\mjh2241\Documents\GitHub\mapTiles\data\3-pmtiles\tilejson.json

Complete output summary:
  addresses.pmtiles (0.0 MB)
  base.pmtiles (3853.1 MB)
  buildings.pmtiles (3651.7 MB)
  divisions.pmtiles (356.9 MB)
  places.pmtiles (28.0 MB)
  transportation.pmtiles (987.8 MB)
  tilejson.json

Total PMTiles size: 8877.5 MB
All files location: C:\Users\mjh2241\Documents\GitHub\mapTiles\data\3-pmtiles


## Using Cartography Properties in Map Styles

The cartography and level properties extracted from Overture Maps can be used in your MapLibre style for dynamic rendering. Here are some examples:

### 1. Zoom-Dependent Filtering
Use `min_zoom` and `max_zoom` to show features only at appropriate zoom levels:

```json
{
  "id": "land-cover-optimized",
  "type": "fill",
  "source": "land-cover-tiles",
  "source-layer": "land-cover",
  "filter": [
    "all",
    [">=", ["zoom"], ["get", "min_zoom"]],
    ["<=", ["zoom"], ["get", "max_zoom"]]
  ],
  "paint": {
    "fill-color": ["get", "color"]
  }
}
```

### 2. Z-Ordering with sort_key
Use `sort_key` for proper feature layering (lower values draw first):

```json
{
  "id": "infrastructure-sorted",
  "type": "fill",
  "source": "infrastructure-tiles",
  "source-layer": "infrastructure",
  "layout": {
    "fill-sort-key": ["get", "sort_key"]  // Higher sort_key = drawn on top
  }
}
```

### 3. Multi-Level Buildings
Filter buildings by floor/level:

```json
{
  "id": "ground-floor-buildings",
  "type": "fill-extrusion",
  "source": "buildings-tiles",
  "source-layer": "buildings",
  "filter": ["==", ["get", "level"], 0],  // Only ground floor
  "paint": {
    "fill-extrusion-height": ["get", "height"],
    "fill-extrusion-base": 0
  }
}
```

### 4. Combined Approach
Combine properties for sophisticated rendering:

```json
{
  "id": "land-use-advanced",
  "type": "fill",
  "source": "land-use-tiles",
  "source-layer": "land-use",
  "filter": [
    "all",
    [">=", ["zoom"], ["coalesce", ["get", "min_zoom"], 0]],  // Fallback to 0
    ["<=", ["zoom"], ["coalesce", ["get", "max_zoom"], 22]]  // Fallback to 22
  ],
  "layout": {
    "fill-sort-key": ["coalesce", ["get", "sort_key"], 0]
  },
  "paint": {
    "fill-opacity": [
      "interpolate", ["linear"], ["zoom"],
      ["coalesce", ["get", "min_zoom"], 8], 0,     // Fade in at min_zoom
      ["+", ["coalesce", ["get", "min_zoom"], 8], 1], 1  // Full opacity 1 zoom level later
    ]
  }
}
```

**Note:** Properties are automatically preserved through the conversion pipeline:
- `tileQueries.sql` extracts properties from Overture Maps Parquet
- `convertToFlatGeobuf.py` preserves all attributes during conversion
- `tippecanoe.py` includes properties in PMTiles output
- MapLibre can access properties via `["get", "property_name"]` expressions